In [1]:
import numpy as np
import cv2
import torch 
import torchvision 
from engine import train_one_epoch, evaluate
from data_reading import WindowDataset
import utils
from torch.utils.data import DataLoader
import itertools
from torch.utils.tensorboard import SummaryWriter


In [2]:
def get_model():
    # Initialize Model, from pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    num_classes = 2 # window + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features # Number of input features for the classifier
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model.double()

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on device {device}')
model = get_model()
model.to(device)

running on device cuda


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr = 0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [5]:
dataset_train = WindowDataset()
dataset_val = WindowDataset(images_folder="data/val/images/", labels_path="data/val/anno")
dataset_test = WindowDataset(images_folder="data/test/images/", labels_path="data/test/anno")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [6]:
train = DataLoader(dataset_train, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
print(next(iter(train)))

val = DataLoader(dataset_val, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
test = DataLoader(dataset_test, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)

((tensor([[[1.0000, 1.0000, 1.0000,  ..., 0.8863, 0.7843, 0.8235],
         [1.0000, 0.9961, 1.0000,  ..., 0.8980, 0.7961, 0.8078],
         [0.9961, 0.9922, 0.9922,  ..., 0.9176, 0.8196, 0.7843],
         ...,
         [0.2353, 0.2510, 0.2706,  ..., 0.7020, 0.7255, 0.7216],
         [0.2588, 0.2667, 0.2784,  ..., 0.6902, 0.7176, 0.7294],
         [0.2392, 0.2392, 0.2627,  ..., 0.6745, 0.7020, 0.7255]],

        [[0.9961, 0.9961, 1.0000,  ..., 0.9294, 0.8471, 0.8902],
         [0.9961, 0.9922, 0.9961,  ..., 0.9412, 0.8588, 0.8745],
         [0.9922, 0.9882, 0.9882,  ..., 0.9608, 0.8706, 0.8471],
         ...,
         [0.3020, 0.3176, 0.3255,  ..., 0.6980, 0.7216, 0.7176],
         [0.3137, 0.3216, 0.3333,  ..., 0.6863, 0.7137, 0.7255],
         [0.2941, 0.2941, 0.3137,  ..., 0.6706, 0.6980, 0.7216]],

        [[0.9882, 0.9804, 0.9843,  ..., 0.9059, 0.8078, 0.8510],
         [0.9882, 0.9765, 0.9765,  ..., 0.9176, 0.8196, 0.8353],
         [0.9843, 0.9725, 0.9686,  ..., 0.9373, 0.8353, 

In [7]:
print(next(iter(train))[1][0]['labels'].dtype)

torch.int64


In [8]:
print(device)
evaluate(model, train, device, print_freq=10)

cuda
Test:  [  0/612]  eta: 0:38:09  model_time: 3.7403 (3.7403)  evaluator_time: 0.0000 (0.0000)  time: 3.7403  data: 0.0000  max mem: 999
Test:  [ 10/612]  eta: 0:23:55  model_time: 2.2411 (2.3790)  evaluator_time: 0.0000 (0.0057)  time: 2.3847  data: 0.0000  max mem: 999
Test:  [ 20/612]  eta: 0:23:01  model_time: 2.2411 (2.3263)  evaluator_time: 0.0000 (0.0046)  time: 2.2635  data: 0.0029  max mem: 999
Test:  [ 30/612]  eta: 0:22:22  model_time: 2.2369 (2.2996)  evaluator_time: 0.0000 (0.0043)  time: 2.2643  data: 0.0048  max mem: 999
Test:  [ 40/612]  eta: 0:21:49  model_time: 2.2282 (2.2811)  evaluator_time: 0.0000 (0.0044)  time: 2.2428  data: 0.0050  max mem: 999
Test:  [ 50/612]  eta: 0:21:21  model_time: 2.2254 (2.2712)  evaluator_time: 0.0000 (0.0047)  time: 2.2367  data: 0.0039  max mem: 999
Averaged stats: model_time: 2.2254 (2.2712)  evaluator_time: 0.0000 (0.0047)
Accumulating evaluation results...
DONE (t=0.03s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.9

In [ ]:
num_epochs = 40
writer = SummaryWriter()
for epoch in range(num_epochs):
    # Train for one epoch, while printing every 10 iterations
    train_one_epoch(writer, model, optimizer, train, device, epoch, print_freq=100)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, val, device, print_freq=100)

print("DONE")

Epoch: [0]  [  0/612]  eta: 0:59:09  lr: 0.000010  loss: 0.7110 (0.7110)  loss_classifier: 0.6892 (0.6892)  loss_box_reg: 0.0001 (0.0001)  loss_objectness: 0.0165 (0.0165)  loss_rpn_box_reg: 0.0052 (0.0052)  time: 5.8006  data: 0.0000  max mem: 2044
Epoch: [0]  [100/612]  eta: 0:49:21  lr: 0.000010  loss: 0.0162 (0.0949)  loss_classifier: 0.0033 (0.0348)  loss_box_reg: 0.0001 (0.0021)  loss_objectness: 0.0021 (0.0330)  loss_rpn_box_reg: 0.0075 (0.0251)  time: 5.7822  data: 0.0016  max mem: 2887
Epoch: [0]  [200/612]  eta: 0:39:43  lr: 0.000010  loss: 0.0151 (0.0777)  loss_classifier: 0.0057 (0.0280)  loss_box_reg: 0.0000 (0.0107)  loss_objectness: 0.0030 (0.0218)  loss_rpn_box_reg: 0.0043 (0.0172)  time: 5.7853  data: 0.0041  max mem: 2887
Epoch: [0]  [300/612]  eta: 0:29:58  lr: 0.000010  loss: 0.0175 (0.0700)  loss_classifier: 0.0020 (0.0237)  loss_box_reg: 0.0001 (0.0134)  loss_objectness: 0.0069 (0.0184)  loss_rpn_box_reg: 0.0067 (0.0145)  time: 5.7848  data: 0.0039  max mem: 2887


In [ ]:
model.eval()
img, labs = next(iter(train))
print(labs)
print(img[0].shape)
img = tuple([i.to(device) for i in img])

In [ ]:
model(img)